カーネル
https://www.kaggle.com/takuyaiida/upura-kaggle-tutorial-01-first-submission

Qiita
https://qiita.com/upura/items/3c10ff6fed4e7c3d70f0

In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

#load packages
import sys #access to system parameters https://docs.python.org/3/library/sys.html
print("Python version: {}". format(sys.version))

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features
print("pandas version: {}". format(pd.__version__))

import matplotlib #collection of functions for scientific and publication-ready visualization
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np #foundational package for scientific computing
print("NumPy version: {}". format(np.__version__))

import scipy as sp #collection of functions for scientific computing and advance mathematics
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook
print("IPython version: {}". format(IPython.__version__)) 

import sklearn #collection of machine learning algorithms
print("scikit-learn version: {}". format(sklearn.__version__))

#misc libraries
import random
import time


#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)


%matplotlib inline
import numpy as np
import pandas as pd

Python version: 3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 14:00:49) [MSC v.1915 64 bit (AMD64)]
pandas version: 0.25.1
matplotlib version: 3.1.1
NumPy version: 1.15.2
SciPy version: 1.1.0
IPython version: 7.8.0
scikit-learn version: 0.20.1
-------------------------


In [34]:
# データの読み込み

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
data = pd.concat([train, test], sort=True) 
print(train.shape)
print(test.shape)
print(data.shape)
# sortはdefaultでTrueになっているがあえて書いてる。カラム名をソートする。データの順は変わらない

(891, 12)
(418, 11)
(1309, 12)


In [35]:
data.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [36]:
data.columns

Index(['Age', 'Cabin', 'Embarked', 'Fare', 'Name', 'Parch', 'PassengerId',
       'Pclass', 'Sex', 'SibSp', 'Survived', 'Ticket'],
      dtype='object')

In [37]:
# 欠損値チェック

data.isnull().sum()

Age             263
Cabin          1014
Embarked          2
Fare              1
Name              0
Parch             0
PassengerId       0
Pclass            0
Sex               0
SibSp             0
Survived        418
Ticket            0
dtype: int64

In [38]:
#　欠損値チェックしたので、次に特徴量エンジニアリング

# 欠損値と中央値or平均値で埋める
# 文字列は数値に変換する

In [39]:
data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)

In [40]:
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [41]:
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)

In [42]:
age_avg = data['Age'].mean()
age_std = data['Age'].std()

data['Age'].fillna(np.random.randint(age_avg - age_std, age_avg + age_std), inplace=True)

In [43]:
#  コード確認  Ageを埋めたのはランダム値
print(age_avg)
print(age_std)
np.random.randint(age_avg - age_std, age_avg + age_std)

29.881137667304014
14.413493211271334


23

In [44]:
# 使わない特徴量の削除
delete_columns = ['Name', 'PassengerId', 'SibSp', 'Parch', 'Ticket', 'Cabin']
data.drop(delete_columns, axis = 1, inplace = True) # axis = 1 で列方向

In [45]:
data

,Age,Embarked,Fare,Pclass,Sex,Survived
0,22.0,0,7.2500,3,0,0.0
1,38.0,1,71.2833,1,1,1.0
2,26.0,0,7.9250,3,1,1.0
3,35.0,0,53.1000,1,1,1.0
4,35.0,0,8.0500,3,0,0.0
...,...,...,...,...,...,...
413,34.0,0,8.0500,3,0,NaN
414,39.0,1,108.9000,1,1,NaN
415,38.5,0,7.2500,3,0,NaN
416,34.0,0,8.0500,3,0,NaN


In [46]:
# concatで統合したやつを元に戻す

train = data[:len(train)]
test = data[len(train):]

In [47]:
y_train = train['Survived']
# y_testはない　予測したいやつなので
X_train = train.drop('Survived', axis = 1)
X_test = test.drop('Survived', axis = 1)

In [48]:
X_train.head()

,Age,Embarked,Fare,Pclass,Sex
0,22.0,0,7.2500,3,0
1,38.0,1,71.2833,1,1
2,26.0,0,7.9250,3,1
3,35.0,0,53.1000,1,1
4,35.0,0,8.0500,3,0


In [49]:
# 機械学習アルゴリズムの設定

In [50]:
from sklearn.linear_model import LogisticRegression

In [51]:
clf = LogisticRegression(penalty='l2', solver="sag", random_state=0)

In [52]:
# 学習 clf.fit
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='sag',
          tol=0.0001, verbose=0, warm_start=False)

In [53]:
# 予測 clf.predict
y_pred = clf.predict(X_test)

In [54]:
y_pred[:20]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0.])

In [55]:
# 提出形式に沿ってcsvファイルを生成して提出用ファイルを作る　以上で終了

sub = pd.DataFrame(pd.read_csv("test.csv")['PassengerId'])
sub['Survived'] = list(map(int, y_pred))
sub.to_csv("submission.csv", index = False)